# Make sure you Alpaca credentials are being read properly and you are connecting to Alpaca

In [ ]:
# Standard libraries
from datetime import datetime, timedelta
import time
import sys

# Non-standard libraries
import pandas as pd
from tqdm import tqdm

# Alpaca API
from alpaca_trade_api.rest import REST

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Project-specific imports
sys.path.append(r"d:\dev\stat_656_autotrader")
from credentials import API_KEY, SECRET_KEY, ENDPOINT_URL 

## Remove the '#' and uncomment the code to print your credentials from the .secrets file

In [ ]:
#print(f"API_KEY: {API_KEY}, SECRET_KEY: {SECRET_KEY}, ENDPOINT_URL: {ENDPOINT_URL}")

## Connect to Alpaca

In [ ]:
# Initialize Alpaca REST client
try:
    alpaca = REST(API_KEY, SECRET_KEY, base_url=ENDPOINT_URL)
    print("Connected to Alpaca successfully!")

except Exception as e:
    print(f"Connection failed: {e}")

## Get a list of all Alpaca stock tickers traded on (NASDAQ, NYSE, AMEX)

In [ ]:
# API call for Alpaca assets
assets = alpaca.list_assets(status='active')  # Consider only active assets

# All US exchanges (NASDAQ, NYSE)
us_exchanges = ['NASDAQ', 'NYSE', 'AMEX']

# Filter to company stocks only (Uppercase letters only, no numbers, periods, underscores, lowercase)
stock_assets = [
    asset for asset in assets
    if asset.exchange in us_exchanges
    and asset.tradable
    and asset.status == 'active'
    and asset.symbol.isalpha()
    and asset.symbol.isupper()
]

stock_tickers = [asset.symbol for asset in stock_assets]

print(f"Fetched {len(stock_tickers)} company stock tickers from Alpaca (U.S. exchanges, no CUSIPs/ETFs/SPACs)!")
print("First 20 company stock tickers:", stock_tickers[:20])

    

## Test the fetch time for Alpaca

In [ ]:
years_back = 5  # Number of years of historical data required
start_date = "2002-01-01"  # Start date for data fetching
end_date = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")  # End date (yesterday)
trading_days_back = years_back * 252  # Approximate number of trading days per year (252)

print(f"Fetching OHLC from {start_date} to {end_date} (~threshold: {trading_days_back} days)")

# Initialize dataframe and performance tracking variables
alpaca_df = pd.DataFrame()
start_time = time.time()
missing_data_count = 0  # Counter for tickers with no data
not_enough_time_count = 0  # Counter for tickers with insufficient data
processed_ticker_count = 0  # Counter for processed tickers
subset_size = 175  # Number of tickers to process in this run
subset_tickers = stock_tickers[:subset_size]  # Select the first 50 tickers

# Loop through each ticker to fetch historical stock data from Alpaca
for ticker in tqdm(subset_tickers, desc="Alpaca Download Progress"):
    try:
        processed_ticker_count += 1  # Increment the processed ticker counter
        bars = alpaca.get_bars(ticker, "1Day", start_date, end_date).df  # Fetch daily OHLC data

        # Check if any data was retrieved
        if not bars.empty:
            # Extract relevant OHLC data and reset the DataFrame index
            df = bars[['open', 'high', 'low', 'close']].reset_index()

            # Check if retrieved data covers the required number of years
            if len(df) < trading_days_back:
                missing_data_count += 1
                not_enough_time_count += 1

                #print(f"Not enough trading days. Skipped {ticker} (~{len(df)} < {trading_days_back} days)")
                continue

            # Add ticker and formatted date columns for clarity
            df['ticker'] = ticker
            df['date'] = df['timestamp'].dt.strftime("%Y-%m-%d")

            # Reorder columns
            df = df[['ticker', 'date', 'open', 'high', 'low', 'close']]

            # Append this ticker's data to the main DataFrame
            alpaca_df = pd.concat([alpaca_df, df])
        else:
            # If no data was retrieved, increment the missing data counter
            missing_data_count += 1
            print(f"No data for {ticker}")

        # Wait briefly to prevent API rate limiting
        time.sleep(1)

    except Exception as e:
        # Handle and log any exceptions during data retrieval
        print(f"Error for {ticker}: {e}")
        time.sleep(1)

# Calculate total processing time and performance metrics
total_seconds = time.time() - start_time
seconds_per_ticker = round((total_seconds / len(subset_tickers)), 2)
estimated_total_time = round(seconds_per_ticker * len(stock_tickers) / 60)

# Output performance summary and sample data
print(f"\nFetched {len(alpaca_df)} rows of stock data in {total_seconds:.2f} seconds.")
print(f"Processed {processed_ticker_count} tickers, missing {missing_data_count} tickers, {not_enough_time_count} tickers did not have enough time.")
print(f"Processing time per ticker: {seconds_per_ticker:.2f} seconds. Estimated processing time for all stocks in the dataset: {estimated_total_time} minutes.")
print("\nExample Table Data:\n")
print("Alpaca Sample:", display(alpaca_df.head(30)))
